# Landscape Elevation Connectivity of an entire mesh

Here the goal is to expand on the work done in the previous notebook (2-Landscape_Elevation_Connectivity_of_a_single_point) by running the LEC calculation across an entire mesh.

This notebook will show you how to load a mesh into gLEC and run a LEC calculation for every point.

This notebook includes a lot of code for visualising the mesh, and the point on it. These bits of code are not necessary for using gLEC, so feel free to skip reading those code chunks. They will be marked as:
```python
# VISUALISATION CODE
```

In [ ]:
import meshio
import numpy as np
from multiprocessing import Pool
import time

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.path import Path
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset
import matplotlib.tri as mtri

label_size = 8
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size

%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

from gLEC.gLEC import gLEC

# Load the mesh and create a `gLEC` object

Here we will follow very similar steps as the previous notebook, and so we will skip over the details for brevity.

In [ ]:
infile = "output_data/AUS_LR.vtk"

mesh = meshio.read(infile)

# VISUALISATION CODE
triang = mtri.Triangulation(mesh.points[:,0], mesh.points[:,1], mesh.cells[0].data)
fig, ax = plt.subplots()
ax.triplot(triang, zorder=1)
ax.set_aspect('equal')

In [ ]:
# create a gLEC object
lec_calculator = gLEC(mesh)

# Run the LEC calculation on all points

In the previous notebook, we chose a single point to run the LEC calculation. Now we want to run it on all the points in the mesh, so we can see how the LEC values change in space.

To do this, we need to first do some filtering on our points: we only care about points above sea level. In some cases, the mesh will already have all points above sea-level.

In [ ]:
valid_starting_points = np.nonzero(mesh.point_data['Z'] >= 0)[0]

In [ ]:
print(f"Total possible starting points:  {mesh.points.shape[0]}")
print(f"Starting points above sea level: {valid_starting_points.shape[0]}")

## LEC for all points

Now to run on all points:


In [ ]:
def calc_area(point):
    came_from, cost_so_far, dist_so_far = lec_calculator.cost_search(point)
    
    # Get a list of all the points we visted
    all_visted_points = came_from.keys()

    # We want a list of fully defined triangles - that is, where we visited all 3
    # vertexes of the triangle.
    neightris = []
    for p in all_visted_points:
        # For our current point, find all the triangles it is part of
        neightris.extend(mesh.cells_dict['triangle'][np.where(mesh.cells_dict['triangle']==p)[0]])
    neightris = np.unique(np.array(neightris), axis=0)

    # For each triangle the point is in, see if the other vertexs were visited in all_visited_points
    good_tris = []
    for tri in neightris:
        if all(vertex in all_visted_points for vertex in tri):
            # If all points in the tri have been visited, then it's a 'good tri'
            good_tris.append(tri)
    good_tris = np.array(good_tris)

    def PolyArea(x,y):
        # From https://stackoverflow.com/a/30408825
        return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

    # For each good triangle, calculate the area, and add it to the total
    area = 0.
    for t in good_tris:
        points = mesh.points[t,:]
        area += PolyArea(points[:,0], points[:,1])
        
    return (point, area)

In [ ]:
LEC_vals = []
factor = 0.0

for i, p in enumerate(valid_starting_points):
    
    LEC_vals.append(calc_area(p))
    
    # Print out the percentage complete, every 10% done.
    if i >= valid_starting_points.shape[0] * factor:
        print(f"{factor * 100}% complete")
        factor += 0.1
print("Done")

In [ ]:
mesh.point_data['LEC'] = np.zeros_like(mesh.point_data['Z'])
for point, val in LEC_vals:
    mesh.point_data['LEC'][point] = val
#meshio.write(outfile, mesh)

In [ ]:
fig, ax = plt.subplots(figsize=(12,15))
ax.set_aspect('equal')

_= ax.tripcolor(triang, mesh.point_data['LEC'], edgecolors='none')